In [1]:
class TestManagement():
    def __init__(self):
        self.testOutputZeros = {}
        self.testOutputOnes = {}

    def store_random_data(self, key, randomData):
        self.testOutputZeros[key] = randomData.count('0')
        self.testOutputOnes[key] = randomData.count('1')

    def basic_prediction(self, key):
        predictOneSuccess = self.testOutputOnes[key] / (self.testOutputZeros[key] + self.testOutputOnes[key])
        predictZeroSuccess = 1 - predictOneSuccess
        return max(predictOneSuccess, predictZeroSuccess)

    def check_bias(self, key):
        predictOneSuccess = self.testOutputOnes[key] / (self.testOutputZeros[key] + self.testOutputOnes[key])
        predictZeroSuccess = 1 - predictOneSuccess
        print("'1' character percentage:", predictOneSuccess)
        print("'0' character percentage:", predictZeroSuccess)
            


def CrudePRNG_output(self, seed=0, length=100):
    nextBit = seed%2
    if (nextBit == 1):
        output = "10" * int(length/2)
    else:
        output = "01" * int(length/2)

    if (length%2 == 1):
        output += str(nextBit)
        if (nextBit == 0): nextBit = 1
        else: nextBit = 0
    
    return output


In [ ]:
import numpy as np
from keras.layers import LSTM, Dense
from keras.models import Sequential

# Training data parameters
num_samples = 10000
sequence_length = 99

# List used to seed the PRNG
seeds = np.random.randint(1000, size=(num_samples, 1))

# Produces training data
fullOutput = []
for seed in seeds:
  fullOutput.append(CrudePRNG_output(seed, sequence_length))

visibleOutput = []
for output in fullOutput:
  visibleOutput.append(output[:-1])

predictionBool = []
for output in fullOutput:
  predictionBool.append(list(output[-1:]))


# Convert the list of strings into a numpy array
X_train = np.array([[int(c) for c in string] for string in visibleOutput])
# Reshape the array to match the input shape expected by the model
X_train = X_train.reshape(len(visibleOutput), sequence_length, 1)


# Convert the list of strings into a numpy array
y_train = np.array([[int(c) for c in string] for string in predictionBool])
# Reshape the array to match the input shape expected by the model
y_train = y_train.reshape(len(predictionBool), 1, 1)


# Produces testing data 
test_samples = 4000

seeds = np.random.randint(1000, size=(test_samples, 1))

fullOutput = []
for seed in seeds:
  fullOutput.append(CrudePRNG_output(seed, sequence_length))

visibleOutput = []
for output in fullOutput:
  visibleOutput.append(output[:-1])

predictionBool = []
for output in fullOutput:
  predictionBool.append(list(output[-1:]))


# Convert the list of strings into a numpy array
X_test = np.array([[int(c) for c in string] for string in visibleOutput])
# Reshape the array to match the input shape expected by the model
X_test = X_test.reshape(len(visibleOutput), sequence_length, 1)


# Convert the list of strings into a numpy array
y_test = np.array([[int(c) for c in string] for string in predictionBool])

# Reshape the array to match the input shape expected by the model
y_test = y_test.reshape(len(predictionBool), 1, 1)


# Sets the model type and sets the layers
model = Sequential()
model.add(LSTM(32, input_shape=(sequence_length, 1), return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

# Compiles the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model with the training data
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evalutes model with test data
loss, accuracy = model.evaluate(X_test, y_test)


Epoch 1/10
313/313 [==============================] - 9s 24ms/step - loss: 0.0479 - accuracy: 0.9904
Epoch 2/10
313/313 [==============================] - 7s 22ms/step - loss: 4.9205e-04 - accuracy: 1.0000
Epoch 3/10
313/313 [==============================] - 7s 22ms/step - loss: 2.5262e-04 - accuracy: 1.0000
Epoch 4/10
313/313 [==============================] - 7s 22ms/step - loss: 1.5638e-04 - accuracy: 1.0000
Epoch 5/10
313/313 [==============================] - 7s 22ms/step - loss: 1.0634e-04 - accuracy: 1.0000
Epoch 6/10
313/313 [==============================] - 7s 22ms/step - loss: 7.6588e-05 - accuracy: 1.0000
Epoch 7/10
313/313 [==============================] - 7s 22ms/step - loss: 5.7342e-05 - accuracy: 1.0000
Epoch 8/10
313/313 [==============================] - 7s 23ms/step - loss: 4.4141e-05 - accuracy: 1.0000
Epoch 9/10
313/313 [==============================] - 7s 22ms/step - loss: 3.4688e-05 - accuracy: 1.0000
Epoch 10/10
125/125 [==============================] - 1s 6